En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

# Librerias y path

In [4]:
import emoji
from collections import Counter
import gc

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
from pandas import json_normalize
import json

#file_path = "farmers-protest-tweets-2021-2-4.json"
file_path2 = "C:\\Users\\JuanJo\\Desktop\\LATAM\\Challenge\\farmers-protest-tweets-2021-2-4.json"



# Dataframes

### df

Tweet original

In [13]:
df0=pd.read_json(file_path2, lines=True)
df=df0.dropna(axis=1, how='all')
df['fecha'] = df['date'].dt.strftime('%Y-%m-%d')

### df_users

detalles de cada user

In [10]:
df_users = json_normalize(df['user'])
df_users['fecha'] = df['fecha'].values

In [43]:
df_acotado2=df[['id','media']]
df_acotado3=df[['id','quotedTweet']]
df_acotado4=df[['id','mentionedUsers']]

In [82]:
#dato=df_acotado2.iloc[0].media

### df_media
detalles de informacion tipo media

In [5]:
df_expandido = df_acotado2.explode('media')
df_media = json_normalize(df_expandido['media'])
df_media['id_prim'] = df_expandido['id'].values
df_media=df_media.dropna(axis=1, how='all')
df_media2 = df_media[df_media['type'].notnull()]

### df_quotedTweet

reTweetos

In [42]:
df_quotedTweet = json_normalize(df['quotedTweet'])
df_quotedTweet['id_prim'] = df_acotado3['id'].values
df_quotedTweet2 = df_quotedTweet[df_quotedTweet['id'].notnull()]
df_quotedTweet3=df_quotedTweet2.dropna(axis=1, how='all')
df_quotedTweet3['fecha'] = pd.to_datetime(df_quotedTweet3['date']).dt.strftime('%Y-%m-%d')


C:\Users\JuanJo\AppData\Local\Temp\ipykernel_17556\1096978238.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quotedTweet3['fecha'] = pd.to_datetime(df_quotedTweet3['date']).dt.strftime('%Y-%m-%d')


### df_mentionedUsers

Usuarios mencionados en tweet original

In [41]:
df_expandido = df_acotado4.explode('mentionedUsers')
df_mentionedUsers = json_normalize(df_expandido['mentionedUsers'])
df_mentionedUsers['id_prim'] = df_expandido['id'].values
df_mentionedUsers=df_mentionedUsers.dropna(axis=1, how='all')
df_mentionedUsers2 = df_mentionedUsers[df_mentionedUsers['id'].notnull()]

### df_mentionedUsers_quot

usuarios mencionados en retweeteos

In [44]:
#df_quotedTweet3_acotado=df_quotedTweet3[['id','id_prim','user.username','user.id','mentionedUsers']]
df_quotedTweet3_acotado=df_quotedTweet3[['id','mentionedUsers']]
df_quotedTweet3_acotado = df_quotedTweet3_acotado[df_quotedTweet3_acotado['mentionedUsers'].notnull()]

df_expandido = df_quotedTweet3_acotado.explode('mentionedUsers')
df_mentionedUsers_quot = json_normalize(df_expandido['mentionedUsers'])
df_mentionedUsers_quot['id_second'] = df_expandido['id'].values
df_mentionedUsers_quot=df_mentionedUsers_quot.dropna(axis=1, how='all')

#dato=df_quotedTweet3_acotado.iloc[0].mentionedUsers

## Pruebas seguimiento de datos

In [9]:
dato0_df=df_quotedTweet3.loc[0]
dato0_idprim=df_quotedTweet3.iloc[0].id_prim
dato0_id=int(df_quotedTweet3.iloc[0].id)
dato0_2_comp=dato0_df.mentionedUsers

dato2_df=df[df['id']==dato0_idprim].loc[0]


dato2_2_df=df_mentionedUsers2[df_mentionedUsers2['id_prim']==dato0_idprim]
dato0_2_df=df_mentionedUsers_quot[df_mentionedUsers_quot['id_second']==dato0_id]



# Pregunta 1

##### a- top10 fechas con más tweets
###### a- Considerando los retweets un tweet por si mismo se hace la suma de los ids por fechas de los tweet y los retweets y se rankea fechas principales.

##### b- usuarios con más publicaciones dichos dias
###### b- Con estas fechas se cuentan los usarios que emitieron tweets y retweets y se obtiene el maximo usuario para estas 10 fechas.

In [36]:
df_agrup = df.groupby(df['fecha'])['id'].count()
#print(len(df[df['fecha']=='2021-02-23'])) ##comprobacion
df_retw_agrup = df_quotedTweet3.groupby(df_quotedTweet3['fecha'])['id'].count()

In [38]:
# Fusionar los DataFrames en función de las fechas
df_merged = pd.merge(df_agrup, df_retw_agrup, on='fecha', how='outer', suffixes=('_dftweets', '_dfretweets'))
df_merged['cantidad_total'] = df_merged['id_dftweets'].fillna(0) + df_merged['id_dfretweets'].fillna(0)

df_merged_sorted = df_merged.sort_values(by='cantidad_total', ascending=False)
top_10_fechas = df_merged_sorted.head(10)


#### Caso especifico para una fecha

In [ ]:
df_user_agrup_0 = df_users.groupby(df_users[df_users['fecha']==top_10_fechas.index[0]]['username'])['id'].count()#.sum() #calza
#print(len(df_users[df_users['fecha']==top_10_fechas.index[0]]))


df_retwet_agrup_0 = df_quotedTweet3.groupby(df_quotedTweet3[df_quotedTweet3['fecha']==top_10_fechas.index[0]]['user.username'])['user.id'].count()#.sum() #calza
#df_retwet_agrup_0=df_retwet_agrup_0.rename({'user.id':'id'})

df_retwet_agrup_0.rename("id",inplace=True)
df_retwet_agrup_0.index.name="username"
#df_retwet_agrup_0.rename("username",axis='index',inplace=True)
#print(len(df_quotedTweet3[df_quotedTweet3['fecha']==top_10_fechas.index[0]]))

In [ ]:

df_merged_users = pd.merge(df_user_agrup_0, df_retwet_agrup_0, on='username', how='outer', suffixes=('_dftweets', '_dfretweets'))
df_merged_users ['cantidad_total'] = df_merged_users ['id_dftweets'].fillna(0) + df_merged_users ['id_dfretweets'].fillna(0)

df_merged_users_sorted = df_merged_users .sort_values(by='cantidad_total', ascending=False)
top_1 = df_merged_users_sorted.head(1)


### Ciclo for para respuesta

In [45]:
P1_data = {'Fecha': [], 'Usuario':[]}
P1 = pd.DataFrame(data=P1_data)


for i in top_10_fechas.index:

    df_user_agrup_0 = df_users.groupby(df_users[df_users['fecha']==i]['username'])['id'].count()
    df_retwet_agrup_0 = df_quotedTweet3.groupby(df_quotedTweet3[df_quotedTweet3['fecha']==i]['user.username'])['user.id'].count()
    df_retwet_agrup_0.rename("id",inplace=True)
    df_retwet_agrup_0.index.name="username"
    df_merged_users = pd.merge(df_user_agrup_0, df_retwet_agrup_0, on='username', how='outer', suffixes=('_dftweets', '_dfretweets'))
    df_merged_users ['cantidad_total'] = df_merged_users ['id_dftweets'].fillna(0) + df_merged_users ['id_dfretweets'].fillna(0)
    df_merged_users_sorted = df_merged_users .sort_values(by='cantidad_total', ascending=False)
    top_1 = df_merged_users_sorted.head(1)
    P1.loc[len(P1)] = np.array([i,top_1.index[0]])

In [46]:
P1
P1['Fecha'] = pd.to_datetime(P1['Fecha'])

In [52]:
output_list_P1 = [(fecha.date(), usuario) for fecha, usuario in zip(P1['Fecha'], P1['Usuario'])]
print(output_list_P1)

[(datetime.date(2021, 2, 12), 'RaviSinghKA'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 14), 'ndtv'), (datetime.date(2021, 2, 16), 'sikh_coalition'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'RaviSinghKA'), (datetime.date(2021, 2, 19), 'Preetm91'), (datetime.date(2021, 2, 23), 'Kisanektamorcha'), (datetime.date(2021, 2, 20), 'MangalJ23056160')]


In [54]:
del P1, P1_data
gc.collect()

38

## Version mas acotada

In [1]:
def rank_users(df_fechas,df_users,df_retweets):
    P1_data = {'Fecha': [], 'Usuario':[]}
    P1 = pd.DataFrame(data=P1_data)
    for i in df_fechas.index:
        df_user_agrup = df_users.groupby(df_users[df_users['date']==i]['username'])['id'].count()
        df_retwet_agrup = df_retweets.groupby(df_retweets[df_retweets['date']==i]['user.username'])['user.id'].count()
        df_retwet_agrup.rename("id",inplace=True)
        df_retwet_agrup.index.name="username"
        df_merged_users = pd.merge(df_user_agrup, df_retwet_agrup, on='username', how='outer', suffixes=('_dftweets', '_dfretweets'))
        df_merged_users ['cantidad_total'] = df_merged_users ['id_dftweets'].fillna(0) + df_merged_users ['id_dfretweets'].fillna(0)
        top_1 = df_merged_users.sort_values(by='cantidad_total', ascending=False).head(1)
        P1.loc[len(P1)] = np.array([i,top_1.index[0]])    
    P1['Fecha'] = pd.to_datetime(P1['Fecha'])
    return P1

In [5]:
cols = ['date','id','user','quotedTweet']
data = []
file_name = file_path2

with open(file_name) as f:
    for line in f:
        doc = json.loads(line)
        lst = [doc['date'], doc['id'],doc['user'],doc['quotedTweet']]
        data.append(lst)

df_P1 = pd.DataFrame(data=data, columns=cols)
df_P1['date'] = pd.to_datetime(df_P1['date']).dt.strftime('%Y-%m-%d')
df_P1['id'] = df_P1['id'].astype('int8') #MEM
cols.clear() #MEM
data.clear() #MEM

In [6]:
df_P1_quotedTweet = json_normalize(df_P1['quotedTweet'])[['id','date','user.id','user.username']]
df_P1_quotedTweet = df_P1_quotedTweet[df_P1_quotedTweet['id'].notnull()]
df_P1_quotedTweet['id'] = df_P1_quotedTweet['id'].astype('int8')#MEM
df_P1_quotedTweet['date'] = pd.to_datetime(df_P1_quotedTweet['date']).dt.strftime('%Y-%m-%d')

In [7]:
df_P1_users = json_normalize(df_P1['user'])[['id','username']]
df_P1_users['date'] = df_P1['date'].values
df_P1_users['id'] = df_P1_users['id'].astype('int8')#MEM

In [8]:
df_P1_agrup = df_P1.groupby(df_P1['date'])['id'].count()
df_P1_retw_agrup = df_P1_quotedTweet.groupby(df_P1_quotedTweet['date'])['id'].count()

del df_P1 #MEM
gc.collect() #MEM

0

In [9]:
df_P1_merged = pd.merge(df_P1_agrup, df_P1_retw_agrup, on='date', how='outer', suffixes=('_dftweets', '_dfretweets'))
df_P1_merged['cantidad_total'] = df_P1_merged['id_dftweets'].fillna(0) + df_P1_merged['id_dfretweets'].fillna(0)
df_P1_merged = df_P1_merged.sort_values(by='cantidad_total', ascending=False).head(10)

del df_P1_agrup, df_P1_retw_agrup #MEM
gc.collect() #MEM

0

In [10]:
P1_short = rank_users(df_P1_merged,df_P1_users,df_P1_quotedTweet)

del df_P1_users,df_P1_quotedTweet,df_P1_merged #MEM
gc.collect() #MEM

0

In [11]:
output_list_P1 = [(fecha.date(), usuario) for fecha, usuario in zip(P1_short['Fecha'], P1_short['Usuario'])]
del P1_short #MEM
gc.collect()
print(output_list_P1)

[(datetime.date(2021, 2, 12), 'RaviSinghKA'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 14), 'ndtv'), (datetime.date(2021, 2, 16), 'sikh_coalition'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'RaviSinghKA'), (datetime.date(2021, 2, 19), 'Preetm91'), (datetime.date(2021, 2, 23), 'Kisanektamorcha'), (datetime.date(2021, 2, 20), 'MangalJ23056160')]


# Pregunta 2

##### a- top10 emojis más usados
###### a- Considerando los retweets un tweet por si mismo se busca en cada content la existencia de emojis y se cuentan.

##### b- conteo de estos emojis
###### b- Se hace la suma general de todo por emoji y se rankea

In [18]:
emojis = []
for i in df['content']:
    emojis += [c for c in i if c in emoji.EMOJI_DATA]
conteo_emojis = Counter(emojis)
df_emojis_twet = pd.DataFrame(list(conteo_emojis.items()), columns=['emoji', 'cantidad'])
df_emojis_twet.set_index('emoji', inplace=True)
#print(df_emojis_twet)

emojis_ret = []
for i in df_quotedTweet3['content']:
    emojis_ret += [c for c in i if c in emoji.EMOJI_DATA]
conteo_emojis = Counter(emojis_ret)
df_emojis_retwet = pd.DataFrame(list(conteo_emojis.items()), columns=['emoji', 'cantidad'])
df_emojis_retwet.set_index('emoji', inplace=True)
#print(df_emojis_retwet)

In [19]:
df_merged_emoji = pd.merge(df_emojis_twet, df_emojis_retwet, on='emoji', how='outer', suffixes=('_dftweets', '_dfretweets'))
df_merged_emoji ['cantidad_total'] = df_merged_emoji ['cantidad_dftweets'].fillna(0) + df_merged_emoji ['cantidad_dfretweets'].fillna(0)

df_merged_emoji_sorted = df_merged_emoji.sort_values(by='cantidad_total', ascending=False)
top_10_emoji = df_merged_emoji_sorted.head(10)


In [20]:
top_10_emoji=top_10_emoji.reset_index()

In [21]:
P2=top_10_emoji[['emoji','cantidad_total']]
P2['cantidad_total'] = P2['cantidad_total'].astype(int)

C:\Users\JuanJo\AppData\Local\Temp\ipykernel_17556\3427506781.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  P2['cantidad_total'] = P2['cantidad_total'].astype(int)


In [22]:
output_list_P2 = [(emo, int(cantidad)) for emo, cantidad in zip(P2['emoji'], P2['cantidad_total'])]
print(output_list_P2)

[('🙏', 8916), ('😂', 4067), ('🚜', 3274), ('✊', 3046), ('🏻', 2696), ('🌾', 2652), ('❤', 2263), ('🤣', 2110), ('👉', 1895), ('👇', 1691)]


## Version mas acotada

In [33]:
cols = ['content','quotedTweet']
data = []
file_name = file_path2

with open(file_name) as f:
    for line in f:
        doc = json.loads(line)
        lst = [doc['content'], doc['quotedTweet']]
        data.append(lst)

df_P2 = pd.DataFrame(data=data, columns=cols)
cols.clear() #MEM
data.clear() #MEM

In [34]:
df_P2_quotedTweet = json_normalize(df_P2['quotedTweet'])[['content']]
df_P2_quotedTweet = df_P2_quotedTweet[df_P2_quotedTweet['content'].notnull()]

In [35]:
emojis_P2 = []
for i in df_P2['content']:
    emojis_P2 += [c for c in i if c in emoji.EMOJI_DATA]
conteo_emojis = Counter(emojis_P2)
df_P2_emojis_twet = pd.DataFrame(list(conteo_emojis.items()), columns=['emoji', 'cantidad'])
df_P2_emojis_twet.set_index('emoji', inplace=True)

emojis_P2.clear()#MEM
conteo_emojis.clear()#MEM
del df_P2#MEM
gc.collect() #MEM

emojis_P2_ret = []
for i in df_P2_quotedTweet['content']:
    emojis_P2_ret += [c for c in i if c in emoji.EMOJI_DATA]
conteo_emojis = Counter(emojis_P2_ret)
df_P2_emojis_retwet = pd.DataFrame(list(conteo_emojis.items()), columns=['emoji', 'cantidad'])
df_P2_emojis_retwet.set_index('emoji', inplace=True)

emojis_P2_ret.clear()#MEM
conteo_emojis.clear()#MEM
del df_P2_quotedTweet#MEM
gc.collect() #MEM

0

In [36]:
df_P2_merged_emoji = pd.merge(df_P2_emojis_twet, df_P2_emojis_retwet, on='emoji', how='outer', suffixes=('_dftweets', '_dfretweets'))
df_P2_merged_emoji ['cantidad_total'] = df_P2_merged_emoji ['cantidad_dftweets'].fillna(0) + df_P2_merged_emoji ['cantidad_dfretweets'].fillna(0)
df_P2_merged_emoji = df_P2_merged_emoji.sort_values(by='cantidad_total', ascending=False).head(10).reset_index()

del df_P2_emojis_twet,df_P2_emojis_retwet#MEM
gc.collect() #MEM

31

In [37]:
P2_short=df_P2_merged_emoji[['emoji','cantidad_total']]
P2_short['cantidad_total'] = P2_short['cantidad_total'].astype(int)

del df_P2_merged_emoji #MEM
gc.collect() #MEM

C:\Users\JuanJo\AppData\Local\Temp\ipykernel_17556\3829611335.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  P2_short['cantidad_total'] = P2_short['cantidad_total'].astype(int)


39

In [38]:
output_list_P2 = [(emo, int(cantidad)) for emo, cantidad in zip(P2_short['emoji'], P2_short['cantidad_total'])]
print(output_list_P2)

del P2_short #MEM
gc.collect() #MEM

[('🙏', 8916), ('😂', 4067), ('🚜', 3274), ('✊', 3046), ('🏻', 2696), ('🌾', 2652), ('❤', 2263), ('🤣', 2110), ('👉', 1895), ('👇', 1691)]


31

# Pregunta 3

##### a- top10 historico users con más menciones
###### a- Considerando los retweets un tweet por si mismo se busca en en los dataframes de usarios mencionados y usuarios mencionados en los retweets cada aparicion de usuarios.

##### b- conteo de estas menciones
###### b- Se hace la suma general de todo por usuario y se rankea

In [45]:
df_agrup_hist = df_mentionedUsers2.groupby(df_mentionedUsers2['username'])['id'].count()
#print(len(df[df['fecha']=='2021-02-23'])) ##comprobacion
df_retw_hist_agrup = df_mentionedUsers_quot.groupby(df_mentionedUsers_quot['username'])['id'].count()

In [46]:
df_merged_menc = pd.merge(df_agrup_hist, df_retw_hist_agrup, on='username', how='outer', suffixes=('_dftweets', '_dfretweets'))
df_merged_menc['cantidad_total'] = df_merged_menc['id_dftweets'].fillna(0) + df_merged_menc['id_dfretweets'].fillna(0)

df_merged_menc_sorted = df_merged_menc.sort_values(by='cantidad_total', ascending=False)
top_10_menciones = df_merged_menc_sorted.head(10)


In [47]:
top_10_menciones=top_10_menciones.reset_index()

In [48]:
P3=top_10_menciones[['username','cantidad_total']]

In [49]:
P3
P3['cantidad_total'] = P3['cantidad_total'].astype(int)

C:\Users\JuanJo\AppData\Local\Temp\ipykernel_17556\670438264.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  P3['cantidad_total'] = P3['cantidad_total'].astype(int)


In [50]:
output_list_P3 = [(username, int(cantidad)) for username, cantidad in zip(P3['username'], P3['cantidad_total'])]
print(output_list_P3)

[('narendramodi', 2623), ('Kisanektamorcha', 2045), ('RakeshTikaitBKU', 1848), ('PMOIndia', 1560), ('GretaThunberg', 1274), ('RahulGandhi', 1252), ('rihanna', 1142), ('DelhiPolice', 1134), ('RaviSinghKA', 1127), ('UNHumanRights', 1057)]


## Version mas acotada

In [61]:
cols = ['mentionedUsers','quotedTweet']
data = []
file_name = file_path2

with open(file_name) as f:
    for line in f:
        doc = json.loads(line)
        lst = [doc['mentionedUsers'], doc['quotedTweet']]
        data.append(lst)

df_P3 = pd.DataFrame(data=data, columns=cols)

cols.clear() #MEM
data.clear() #MEM

In [62]:
df_P3_mentionedUsers=df_P3.explode('mentionedUsers')
df_P3_mentionedUsers = json_normalize(df_P3_mentionedUsers['mentionedUsers'])[['id','username']]
df_P3_mentionedUsers = df_P3_mentionedUsers[df_P3_mentionedUsers['id'].notnull()]
df_P3_mentionedUsers['id'] = df_P3_mentionedUsers['id'].astype('int8') #MEM

In [63]:
df_P3_quotedTweet = json_normalize(df_P3['quotedTweet'])[['mentionedUsers']]
df_P3_quotedTweet = df_P3_quotedTweet[df_P3_quotedTweet['mentionedUsers'].notnull()]

df_P3_mentionedUsers_quot=df_P3_quotedTweet.explode('mentionedUsers')
df_P3_mentionedUsers_quot = json_normalize(df_P3_mentionedUsers_quot['mentionedUsers'])[['id','username']]
df_P3_mentionedUsers_quot['id'] = df_P3_mentionedUsers_quot['id'].astype('int8') #MEM

del df_P3_quotedTweet, df_P3 #MEM
gc.collect() #MEM

0

In [64]:
df_P3_agrup_hist = df_P3_mentionedUsers.groupby(df_P3_mentionedUsers['username'])['id'].count()
df_P3_retw_hist_agrup = df_P3_mentionedUsers_quot.groupby(df_P3_mentionedUsers_quot['username'])['id'].count()

del df_P3_mentionedUsers, df_P3_mentionedUsers_quot #MEM
gc.collect() #MEM

31

In [65]:
df_P3_merged_menc = pd.merge(df_P3_agrup_hist, df_P3_retw_hist_agrup, on='username', how='outer', suffixes=('_dftweets', '_dfretweets'))
df_P3_merged_menc['cantidad_total'] = df_P3_merged_menc['id_dftweets'].fillna(0) + df_P3_merged_menc['id_dfretweets'].fillna(0)

df_P3_merged_menc = df_P3_merged_menc.sort_values(by='cantidad_total', ascending=False).head(10).reset_index()

del df_P3_agrup_hist, df_P3_retw_hist_agrup #MEM
gc.collect() #MEM

31

In [66]:
P3_short=df_P3_merged_menc[['username','cantidad_total']]
P3_short['cantidad_total'] = P3_short['cantidad_total'].astype(int)

del df_P3_merged_menc #MEM
gc.collect() #MEM

C:\Users\JuanJo\AppData\Local\Temp\ipykernel_17556\1316740264.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  P3_short['cantidad_total'] = P3_short['cantidad_total'].astype(int)


31

In [67]:
output_list_P3 = [(username, int(cantidad)) for username, cantidad in zip(P3_short['username'], P3_short['cantidad_total'])]
print(output_list_P3)

del P3_short #MEM
gc.collect() #MEM

[('narendramodi', 2623), ('Kisanektamorcha', 2045), ('RakeshTikaitBKU', 1848), ('PMOIndia', 1560), ('GretaThunberg', 1274), ('RahulGandhi', 1252), ('rihanna', 1142), ('DelhiPolice', 1134), ('RaviSinghKA', 1127), ('UNHumanRights', 1057)]


31